In [ ]:
import mysql.connector
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def load_properties(filepath, sep='=', comment_char='#'):
    """
    Read the file passed as parameter as a properties file.
    """
    props = {}
    with open(filepath, "rt") as f:
        for line in f:
            l = line.strip()
            if l and not l.startswith(comment_char):
                key_value = l.split(sep)
                key = key_value[0].strip()
                value = sep.join(key_value[1:]).strip().strip('"') 
                props[key] = value 
    return props

In [ ]:
connection = mysql.connector.connect(**load_properties('./project.properties'))

In [ ]:
### Get values direct from table

query = '''
select date, avg(rainfall) as rain, airtemp_mean as temp from weatherdailymeasurement
  where date >= '2021-12-01'
  -- and date < '2021-02-01'
  group by date'''

sbb_query = '''
select date, zugpuenktlichkeit from sbbdelay
  where date >= '2021-12-01'
  -- and date < '2021-02-01'
  and sbbregion_isocode = 'NETZ'
'''

result = pd.read_sql(sql=query, con=connection)
result_df = pd.DataFrame(result)

sbb_result = pd.read_sql(sql=sbb_query, con=connection)
sbb_result_df = pd.DataFrame(sbb_result)

In [ ]:
### get values from view

query = '''
select * from weatherdailydelay
     
where sbbregion_isocode = 'ROT'
'''

result = pd.read_sql(sql=query, con=connection)
result_df = pd.DataFrame(result)

In [ ]:
### change puenktlichkeit to delay, for better visual understanding of correlation to rain and temperature

result_df.zugpuenktlichkeit = 100 - result_df.zugpuenktlichkeit

result_df.head()

### Lineplot

In [ ]:
sns.lineplot(data=result_df)
# sns.lineplot(data=sbb_result_df)

### Scatterplot

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
custom_cmap = LinearSegmentedColormap.from_list(name="my_cmap", colors=["blue", "red"])

custom_cmap

In [ ]:
sns.scatterplot(x=result_df['rain'], y=sbb_result_df['zugpuenktlichkeit'], c=result_df.temp ,cmap=custom_cmap)

In [ ]:
sns.scatterplot(x=result_df.moving_average, y=result_df.zugpuenktlichkeit, c=result_df.temp ,cmap=custom_cmap)